# **Predicción Ventas de Yamaha 2024-2025**

In [61]:
# Set Project params
data_filter = 'modelo' # [asesor, modelo]
data_values = 'cantidad' # [cantidad, costo]

data_time_start = '2014' # =>  2022-04-01
data_time_end = '2025' # =>  2024-01-01

show_plots = False

## **First Steps**

Set init imports and variables

In [62]:
# Import needed libraries
import pandas
import numpy
import seaborn
from matplotlib import pyplot
%matplotlib inline

In [63]:
# Set the variables according Project params
filter_mappings = {
   'asesor': 'nom_asesor',
   'modelo': 'modelo',
   'clasificacion': 'clasificacion'
}
values_mappings = {
   'cantidad': {
      'name': 'cantidad',
      'type': int
   },
   'costo': {
      'name': 'costo_unitario',
      'type': float
   }
}

selected_filter = filter_mappings.get(data_filter)
selected_value = values_mappings.get(data_values)

## **Get and Prepare Data**

Data connection, cleaning, normalize, filter and sort.

In [64]:
# Set data files paths
import os

query_path = '../assets/query.sql'
normalize_path = f'../assets/{data_filter}.csv'

data_path = '../assets/data.csv'
data_clean_path = '../assets/data_cleaned.csv'
data_normalized_path = '../assets/data_normalized.csv'
data_filter_path = '../assets/data_filtered.csv'

### Connect

In [65]:
# Get connection env variables 
if not os.path.exists(data_path):

   import dotenv

   dotenv.load_dotenv( )

   DRIVER = 'ODBC Driver 18 for SQL Server'
   SERVER = os.getenv('PROJECT_SERVER')
   DATABASE = os.getenv('PROJECT_DATABASE')
   USERNAME = os.getenv('PROJECT_USERNAME')
   PASSWORD = os.getenv('PROJECT_PASSWORD')

In [66]:
# Fetch data from DB
if not os.path.exists(data_path):

   # import pyodbc
   import sqlalchemy
   
   # Get SQL query from file
   with open(query_path) as file:
      sql_query = file.read()

   # PyODBC connection
   # pyodbc_connection_string = f'DRIVER={DRIVER};SERVER={SERVER};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD};TrustServerCertificate=YES;'
   # pyodbc_connection = pyodbc.connect(pyodbc_connection_string)

   # SQLAlquemy connection
   sqlalchemy_connection_string = f'mssql+pyodbc://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}?driver={DRIVER}&TrustServerCertificate=yes'
   sqlalchemy_engine = sqlalchemy.create_engine(sqlalchemy_connection_string)

   # Excecute query with pandas
   query = pandas.read_sql_query(sql_query, sqlalchemy_engine)

In [67]:
# Save data
if not os.path.exists(data_path):   
   
   fetched_data = pandas.DataFrame(query)
   fetched_data.to_csv(data_path, index=False, header=True, sep=";")

### Clean

In [68]:
# Load data to clean
with open(data_path, 'r') as file:
   data = file.read().split('\n')

   headers = data.pop(0)
   clean_data = '\n'.join(data)

   print(data[0] +'\n'+ data[17792] +'\n'+ data[8667] +'\n'+ data[30405])

1;5004;2014-10-01;79544064.0;BELLO VARGAS ESTEBAN;80007136.0;VEGA CAÑON JHON ALFREDO;1.0;4147813.0;938394.0;FAZER ;FZ16 ST FAZER;NA;1.0;VITRINA;HN/11
1;5004;2019-02-16;1016009268.0;FONSECA CASALLAS JULIETH;80069513.0;CARO NEUTO HECTOR JAVIER;1.0;5576470.0;1062185.5;BWSX;YW125X BWS125X;NA;2.0;VITRINA;HN/19488
1;5004;2016-09-30;80435341.0;SOTO CORREA OMAR JOSE;52752007.0;BELTRAN GONZALEZ SANDRA MILENA;1.0;5061724.0;964138.0;FZ-S;FZN150D-6 (FZ-S);NA;7.0;VITRINA;HN/9514
1;8004;2021-12-06;52529586.0;CANTOR SUAREZ LEISDY PAOLA;1038815080.0;MONTOYA MONTERROSA HAROLD ANTONIO;1.0;3571429.0;882353.0;YCZ110;YC110D;NA;66.0;CONVENIO;HN/33714


In [69]:
# Clean data by removing extra characters

# decimals to int values
clean_data = clean_data.replace('.0;', ';')
# extra commas
clean_data = clean_data.replace(',', '')
# double spaces at start and end of any cell
clean_data = clean_data.replace('; ', ';')
clean_data = clean_data.replace(' ;', ';')
# double spaces at middle of any cell
clean_data = clean_data.replace('\n', '_')
clean_data = ' '.join(clean_data.split())
clean_data = clean_data.replace('_', '\n')
# extra quotation marks
clean_data = clean_data.replace('"', '')


clean_data = clean_data.split('\n')
print(clean_data[0] +'\n'+ clean_data[17792] +'\n'+ clean_data[8667] +'\n'+ clean_data[30405])
clean_data = '\n'.join(clean_data)

1;5004;2014-10-01;79544064;BELLO VARGAS ESTEBAN;80007136;VEGA CAÑON JHON ALFREDO;1;4147813;938394;FAZER;FZ16 ST FAZER;NA;1;VITRINA;HN/11
1;5004;2019-02-16;1016009268;FONSECA CASALLAS JULIETH;80069513;CARO NEUTO HECTOR JAVIER;1;5576470;1062185.5;BWSX;YW125X BWS125X;NA;2;VITRINA;HN/19488
1;5004;2016-09-30;80435341;SOTO CORREA OMAR JOSE;52752007;BELTRAN GONZALEZ SANDRA MILENA;1;5061724;964138;FZ-S;FZN150D-6 (FZ-S);NA;7;VITRINA;HN/9514
1;8004;2021-12-06;52529586;CANTOR SUAREZ LEISDY PAOLA;1038815080;MONTOYA MONTERROSA HAROLD ANTONIO;1;3571429;882353;YCZ110;YC110D;NA;66;CONVENIO;HN/33714


In [70]:
# Save cleaned data
headers = headers.replace(';', ',')
clean_data = clean_data.replace(';', ',')

with open(data_clean_path, 'w') as file:
   file.write(headers + '\n' + clean_data)

### Normalize

In [71]:
# Load data to normalize
if os.path.exists(normalize_path):
   normalize_data = pandas.read_csv(data_clean_path)
   
normalize_data.head()

,sw,bodega,fecha,ident_asesor,nom_asesor,ident_cliente,nom_cliente,cantidad,costo_unitario,utilidad,modelo,des_modelo,financiera,dias_inv,clasificacion,doc_ref
0,1,5004,2014-10-01,79544064,BELLO VARGAS ESTEBAN,80007136,VEGA CAÑON JHON ALFREDO,1,4147813.0,938394.00,FAZER,FZ16 ST FAZER,NaN,1.0,VITRINA,HN/11
1,1,5004,2014-10-01,1024511514,VILLARRAGA GARCIA DERLI LORENA,1014222536,RODRIGUEZ CRUZ BORIS SEBASTIAN,1,2557936.0,588615.75,YBR,YBR125ESD,NaN,1.0,VITRINA,HN/5
2,1,3004,2014-10-02,79564916,PAEZ VEGA JUAN CARLOS,1121837576,ROMERO LESMES ROGER JOSUE,1,6577241.0,793448.50,R15,YZF155-A (YZF-R15),NaN,2.0,VITRINA,HN/33
3,1,5004,2014-10-02,79820335,CACERES FRAILE JHON WILIAM,80219236,BARRIGA CUBIDES JULIO CESAR,1,4324466.0,977258.00,XTZ125,XTZ125,NaN,2.0,VITRINA,HN/21
4,1,5004,2014-10-02,79820335,CACERES FRAILE JHON WILIAM,79183832,CARRANZA MONTENEGRO VICTOR FABIAN,1,4147813.0,938394.00,FZ,FZN150D-6 (FZ-S),NaN,0.0,VITRINA,HN/4


In [72]:
# Load normalization params
if os.path.exists(normalize_path):

   with open(normalize_path, mode='r', encoding='utf-8') as file:
      norms = file.read().replace('\n', ',').split(',')
      norms = norms[2:]
      norms = numpy.reshape(norms, (-1, 2))

   # Convert normalization csv to dict
   normalization = {}
   for model, norm_value in norms:
      normalization[model] = norm_value

normalization

{'AF115F FINO': 'FINO115',
 'CZD300-A (X-MAX300)': 'XMAX300',
 'CZD300-A XMAX': 'XMAX300',
 'FZ15N (FZ)': 'FZ150',
 'FZ15S (FAZER)': 'FAZER150',
 'FZ16 ST FAZER': 'FAZER150',
 'FZN 150A': 'FZ150',
 'FZN150D': 'FZ150',
 'FZN150D-6 (FZ-S)': 'FZ150',
 'FZN250-A': 'FZ250',
 'GDR155-A': 'GDR155',
 'GPD150': 'NMAX150',
 'GPD150 (NMAX)': 'NMAX150',
 'GPD150-A (NMAX)': 'NMAX150',
 'GPD155-A (NMAX155)': 'NMAX150',
 'MT09TRA': 'MT09',
 'MT10SP': 'MT10',
 'MTM850 (XSR900)': 'XSR900',
 'MTN1000 (MT10)': 'MT10',
 'MTN155-A': 'MT150',
 'MTN320-A (MT03)': 'MT03',
 'MTN690 (MT07)': 'MT07',
 'MTN890 (MT09)': 'MT09',
 'MTN890D (MT09SP)': 'MT09',
 'MTT850D (MT09TRAGT)': 'MT09',
 'MTT890D (MT09 TRACER GT)': 'MT09',
 'MW125 (TRICITY)': 'TRICITY',
 'SZ15RR': 'SZ150',
 'SZ16R': 'SZ150',
 'T115': 'CRYPTON 115',
 'T115FI': 'CRYPTON 115',
 'TTR50E': 'TTR50',
 'XJ6N': 'XJ6',
 'XP560D (TMAX TECH MAX)': 'TMAX',
 'XSR900': 'TMAX',
 'XT1200 ZE': 'TENERE 1200',
 'XT1200Z': 'TENERE 1200',
 'XT660R': 'XT660',
 'XTZ125'

In [73]:
# Insert normalized data in a new Column
if os.path.exists(normalize_path):

   normalize_data['modelo'] = normalize_data['des_modelo'].apply(lambda x: normalization.get(x))
normalize_data.head()

,sw,bodega,fecha,ident_asesor,nom_asesor,ident_cliente,nom_cliente,cantidad,costo_unitario,utilidad,modelo,des_modelo,financiera,dias_inv,clasificacion,doc_ref
0,1,5004,2014-10-01,79544064,BELLO VARGAS ESTEBAN,80007136,VEGA CAÑON JHON ALFREDO,1,4147813.0,938394.00,FAZER150,FZ16 ST FAZER,NaN,1.0,VITRINA,HN/11
1,1,5004,2014-10-01,1024511514,VILLARRAGA GARCIA DERLI LORENA,1014222536,RODRIGUEZ CRUZ BORIS SEBASTIAN,1,2557936.0,588615.75,LIBERO 125,YBR125ESD,NaN,1.0,VITRINA,HN/5
2,1,3004,2014-10-02,79564916,PAEZ VEGA JUAN CARLOS,1121837576,ROMERO LESMES ROGER JOSUE,1,6577241.0,793448.50,R15,YZF155-A (YZF-R15),NaN,2.0,VITRINA,HN/33
3,1,5004,2014-10-02,79820335,CACERES FRAILE JHON WILIAM,80219236,BARRIGA CUBIDES JULIO CESAR,1,4324466.0,977258.00,XTZ150,XTZ125,NaN,2.0,VITRINA,HN/21
4,1,5004,2014-10-02,79820335,CACERES FRAILE JHON WILIAM,79183832,CARRANZA MONTENEGRO VICTOR FABIAN,1,4147813.0,938394.00,FZ150,FZN150D-6 (FZ-S),NaN,0.0,VITRINA,HN/4


In [74]:
# Save normalized data
if os.path.exists(normalize_path):
   
   normalize_data.to_csv(data_normalized_path, index=False)

### Filter

In [75]:
# Load data to filter
filter_data = pandas.read_csv(data_normalized_path, parse_dates=['fecha'], date_format='%Y-%m-%d')

In [76]:
# Delete unnecesary columns
filter_data = filter_data.drop(columns=['sw', 'bodega', 'ident_asesor', 'ident_cliente', 'nom_cliente', 'utilidad', 'financiera', 'dias_inv', 'doc_ref'])
filter_data.head()

,fecha,nom_asesor,cantidad,costo_unitario,modelo,des_modelo,clasificacion
0,2014-10-01,BELLO VARGAS ESTEBAN,1,4147813.0,FAZER150,FZ16 ST FAZER,VITRINA
1,2014-10-01,VILLARRAGA GARCIA DERLI LORENA,1,2557936.0,LIBERO 125,YBR125ESD,VITRINA
2,2014-10-02,PAEZ VEGA JUAN CARLOS,1,6577241.0,R15,YZF155-A (YZF-R15),VITRINA
3,2014-10-02,CACERES FRAILE JHON WILIAM,1,4324466.0,XTZ150,XTZ125,VITRINA
4,2014-10-02,CACERES FRAILE JHON WILIAM,1,4147813.0,FZ150,FZN150D-6 (FZ-S),VITRINA


In [77]:
# Filter by time-range Project params
filter_data = filter_data[(data_time_start <= filter_data['fecha']) & (filter_data['fecha']<= data_time_end)]
filter_data

,fecha,nom_asesor,cantidad,costo_unitario,modelo,des_modelo,clasificacion
0,2014-10-01,BELLO VARGAS ESTEBAN,1,4147813.0,FAZER150,FZ16 ST FAZER,VITRINA
1,2014-10-01,VILLARRAGA GARCIA DERLI LORENA,1,2557936.0,LIBERO 125,YBR125ESD,VITRINA
2,2014-10-02,PAEZ VEGA JUAN CARLOS,1,6577241.0,R15,YZF155-A (YZF-R15),VITRINA
3,2014-10-02,CACERES FRAILE JHON WILIAM,1,4324466.0,XTZ150,XTZ125,VITRINA
4,2014-10-02,CACERES FRAILE JHON WILIAM,1,4147813.0,FZ150,FZN150D-6 (FZ-S),VITRINA
...,...,...,...,...,...,...,...
41047,2024-05-25,ROMANA CAICEDO ERIC RODRIGO,1,7571429.0,FZ150,FZN 150A,VITRINA
41048,2024-05-27,MONTEJO LONDOÑO DANIEL STIVEN,1,10714000.0,MT150,MTN155-A,VITRINA
41049,2024-05-27,MONTEJO LONDOÑO DANIEL STIVEN,1,11500000.0,NMAX150,GPD155-A (NMAX155),VITRINA
41050,2024-05-27,MONTEJO LONDOÑO DANIEL STIVEN,1,7571429.0,FZ150,FZN 150A,VITRINA


In [78]:
# Group dataframe by ['fecha'] as primary and [selected_filter] as secondary
filter_data_group = filter_data.groupby([pandas.Grouper(key='fecha', freq='D', sort=True), selected_filter])[selected_value['name']].sum()
filter_data_group

fecha       modelo    
2014-10-01  FAZER150      1
            LIBERO 125    1
2014-10-02  FZ150         1
            R15           1
            SZ150         1
                         ..
2024-05-25  R15           2
            XTZ150        5
2024-05-27  FZ150         1
            MT150         1
            NMAX150       0
Name: cantidad, Length: 14934, dtype: int64

In [79]:
# Convert group series in a new dataframe
data_filtered = filter_data_group.unstack(level=1)
data_filtered.head()

modelo,BWIS125,CRYPTON 115,CRYPTON125,FAZER150,FINO115,FZ150,FZ250,GDR155,GRIZZLY 350,LIBERO 125,...,XJ6,XMAX300,XSR900,XT660,XTZ150,XTZ250,XV 950,YCZ110,YZ125,YZ65
fecha,,,,,,,,,,,,,,,,,,,,,
2014-10-01,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-10-02,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2014-10-03,1.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-10-04,1.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-10-06,2.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
# Fill NaN and format int columns
data_filtered = data_filtered.fillna(0)
data_filtered = data_filtered.astype(selected_value['type'])
data_filtered.head()

modelo,BWIS125,CRYPTON 115,CRYPTON125,FAZER150,FINO115,FZ150,FZ250,GDR155,GRIZZLY 350,LIBERO 125,...,XJ6,XMAX300,XSR900,XT660,XTZ150,XTZ250,XV 950,YCZ110,YZ125,YZ65
fecha,,,,,,,,,,,,,,,,,,,,,
2014-10-01,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2014-10-02,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2014-10-03,1,0,0,1,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2014-10-04,1,0,0,0,0,3,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2014-10-06,2,0,0,3,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [81]:
# Create 'TOTAL' by Time
data_filtered['TOTAL'] = data_filtered.sum(axis='columns')
# data_filtered.loc['Total']= data_filtered.sum()

data_filtered

modelo,BWIS125,CRYPTON 115,CRYPTON125,FAZER150,FINO115,FZ150,FZ250,GDR155,GRIZZLY 350,LIBERO 125,...,XMAX300,XSR900,XT660,XTZ150,XTZ250,XV 950,YCZ110,YZ125,YZ65,TOTAL
fecha,,,,,,,,,,,,,,,,,,,,,
2014-10-01,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,2
2014-10-02,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,4
2014-10-03,1,0,0,1,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,5
2014-10-04,1,0,0,0,0,3,0,0,0,1,...,0,0,0,0,0,0,0,0,0,6
2014-10-06,2,0,0,3,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-22,0,1,0,0,0,3,0,0,0,0,...,0,0,0,2,0,0,0,0,0,11
2024-05-23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,7,0,0,0,0,0,13
2024-05-24,0,0,0,0,0,13,1,0,0,0,...,0,0,0,2,0,0,0,0,0,28


In [82]:
# Save filtered_data
data_filtered.to_csv(data_filter_path, date_format='%Y-%m-%d')

### Sort

In [83]:
# Group dataframe by ['modelo'] as primary and ['des_modelo'] as secondary
sort_data_group = filter_data.groupby(['modelo', 'des_modelo'])[selected_value['name']].sum()
sort_data_group

modelo       des_modelo    
BWIS125      YW125               17
             YW125 BWS125       510
             YW125X BWS125X    2032
             YW125XFI          1047
CRYPTON 115  T115                55
                               ... 
XV 950       XVS950CU/ABS         2
YCZ110       YC110D             787
             YC110D (YC-Z)       39
YZ125        YZ125                1
YZ65         YZ65                 1
Name: cantidad, Length: 65, dtype: int64

In [84]:
# Sort Items by Value
sort_data = filter_data.groupby(selected_filter)[selected_value['name']].sum()
sort_data = sort_data.reset_index()
sort_data = sort_data.sort_values(by='cantidad', ascending=False)
sort_data

,modelo,cantidad
5,FZ150,14178
6,FZ250,3975
31,XTZ150,3843
0,BWIS125,3606
17,R15,3432
21,SZ150,2302
15,NMAX150,1952
3,FAZER150,1860
14,MT150,908
34,YCZ110,826


## **Display Data**

Display sample graphs of the prepared data

### Show

In [85]:
# Load data
data = pandas.read_csv('../assets/data_filtered.csv', parse_dates=['fecha'], date_format='%Y-%m-%d', dtype=selected_value['type'])

# Extract Items and Time
items = list(data.iloc[:, 1:-1].keys())
time = numpy.asarray(data['fecha'], dtype='datetime64[s]')

data.head()

,fecha,BWIS125,CRYPTON 115,CRYPTON125,FAZER150,FINO115,FZ150,FZ250,GDR155,GRIZZLY 350,...,XMAX300,XSR900,XT660,XTZ150,XTZ250,XV 950,YCZ110,YZ125,YZ65,TOTAL
0,2014-10-01,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,2014-10-02,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,4
2,2014-10-03,1,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,5
3,2014-10-04,1,0,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,6
4,2014-10-06,2,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7


In [86]:
# Items all-in-one plot
if show_plots:
   
   figure, ax = pyplot.subplots(figsize=(12, 4))

   ax.plot(time, data[items], lw=1)
   ax.tick_params(axis='x', labelrotation=0)
   ax.set_title(f'No. de Ventas {data_time_start} - {data_time_end}')
   ax.set_xlabel('Fecha')
   ax.set_ylabel('Ventas')
   ax.margins(x=0.03, y=0.02)
   ax.grid()

   figure.tight_layout()

### Relations

In [87]:
# Top-10 First & Last items - filtered data 
first_last = list( sort_data['modelo'] )
items_first_last = [ first_last[0], first_last[9] ]

In [88]:
# Top-10 First & Last items VS total sales relation - filtered data 
if show_plots:
   figure_b, axes_b = pyplot.subplots(nrows=2, ncols=1, figsize=(14, 7))

   for index, key in enumerate(items_first_last):
      ax = axes_b[index]
      ax.plot(time, data['TOTAL'], label='Total', lw=1)
      ax.plot(time, data[key], label=key, lw=1)
      ax.tick_params(axis='x', labelrotation=0)
      ax.margins(x=0.03, y=0.04)
      # ax.grid()
      ax.set(
         title=f'{key}',
         # xlabel='Fecha', 
         # ylabel='No. de Ventas',
      )
      ax.legend()

   figure_b.tight_layout()

### Top-10

In [89]:
# Top-10 Items - filtered data
items_top_10 = list( sort_data['modelo'].head(10) )

# Config plots for this Items subgroup
cols = 2
rows = 5
size = (10, 8)
y_limit = (0, data[items].max().max())
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

In [90]:
# Top-10 items same-scaled separated plots
if show_plots:
    figure_a, axes_a = pyplot.subplots(nrows=rows, ncols=cols, figsize=size)

    for index, item in enumerate(items_top_10):
        ax = axes_a[int(index/cols), int(index%cols)]
        ax.plot(time, data[item], label=item, color=colors[int(index%10)], lw=1)
        ax.tick_params(axis='x', labelrotation=0)
        ax.set(ylim=y_limit)
        ax.legend()
        # ax.grid()

    figure_a.tight_layout()